This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [144]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'zHH8pzRSeyofg3bnBAcz'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [145]:
# First, import the relevant modules
import requests
import json
import collections
import pprint


In [146]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
base_request_str = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?&api_key=' + API_KEY
r = requests.get(base_request_str + '&start_date=2018-06-18&end_date=2018-06-18')
one_day = r.json()

In [147]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
pprint.pprint(one_day)

{'dataset_data': {'collapse': None,
                  'column_index': None,
                  'column_names': ['Date',
                                   'Open',
                                   'High',
                                   'Low',
                                   'Close',
                                   'Change',
                                   'Traded Volume',
                                   'Turnover',
                                   'Last Price of the Day',
                                   'Daily Traded Units',
                                   'Daily Turnover'],
                  'data': [['2018-06-18',
                            None,
                            61.55,
                            60.0,
                            60.35,
                            None,
                            77196.0,
                            4663863.0,
                            None,
                            None,
                            None]],


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [148]:
r = requests.get(base_request_str + '&start_date=2017-01-01&end_date=2017-12-31')
json_2017 = r.json()
print(json_2017['dataset_data']['column_names'])
keys = json_2017['dataset_data']['column_names']
data = json_2017['dataset_data']['data']

['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']


In [149]:
data_2017 = dict.fromkeys(keys,[])
for k in range(len(keys)):
    data_2017[keys[k]] = [d[k] for d in data]

In [150]:
min_open = min(x for x in data_2017['Open'] if x is not None)
max_open = max(x for x in data_2017['Open'] if x is not None)

print('The highest opening price was %.2f and the lowest was %0.2f.' %(max_open,min_open))

The highest opening price was 53.11 and the lowest was 34.00.


In [151]:
high_low = list(zip(data_2017['High'],data_2017['Low']))
price_changes = [(h - l) for (h,l) in high_low]
print('The largest change in any one day was %.2f.' %max(price_changes))

The largest change in any one day was 2.81.


In [155]:
high_low_high_low = list(zip(data_2017['High'][0:-1],
                             data_2017['Low'][0:-1],
                             data_2017['High'][1:],
                             data_2017['Low'][1:]))
price_changes_2d = [max(h2 - l1,h1-l2) for (h1,l1,h2,l2) in high_low_high_low]
print('The largest change in any two days was %.2f' %max(price_changes_2d))

The largest change in any two days was 4.30


In [153]:
vol_mean_1 = sum(data_2017['Traded Volume'])/len(data_2017['Traded Volume'])
print('The average daily trading volume was %0.2f.'%vol_mean_1)

The average daily trading volume was 89124.34.


In [154]:
def median(l):
    n = len(l)
    if n == 0:
        return None
    l2 = sorted(l)
    if n%2 == 0:
        mid1 = n//2
        mid2 = mid1 - 1
        return (l2[mid1] + l2[mid2])/2
    else:
        mid = (n-1)//2
        return l2[mid]
        
print('The median daily trading volume in 2017 was %.2f.'%median(data_2017['Traded Volume']))

The median daily trading volume in 2017 was 76286.00.
